# Parameters for simulations
This notebook allows to create a config.json file containing all the parameters for the simulation to be passed to the main script scripts/run_simulations.py

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

In [2]:
# System configuration: edit name of the dynamical system and its parameters
system = {
    "name": "kuramoto",
    "params": {
        "theta": 1.0,
    },
}

# Network configuration: edit name, size = number of particles and parameters of the network
network = {
    "name": "bistable_graphon",
    "params": {
        "n": 500,
    },
}

# Noise configuration
noise = {
    "name": "additive_gaussian",
    "params": {
        "sigma": 0.1,
    },
}

# Integrator configuration
integrator = {
    "tmin": 0.0,
    "tmax": 20000.0,
    "dt": 0.1,
    "stats_every": 10, # every "stats_every" timesteps get the appropriate statistics and write them to file
    "state_every": 10000, # every "state_every" timesteps write the state of the system to file
    "write_stats_at_start": True, 
    "write_state_at_start": True,
}

# Initial condition configuration
initial_condition = {
    "type": "uniform", # choose the type of initial condition + its parameters
    "low": 0.0,
    "high": 2 * np.pi,
}

config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


In [3]:
# The json file will be saved at root_folder/path_folder/name_config_file
path_folder = "configs"
name_config_file = "config_Kuramoto_Bistable_Graphon.json"

# Save it in the configs folder
repo_root = Path.cwd().parent
output_path = repo_root / path_folder / name_config_file
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(config, indent=2))
print(f"Wrote config to {output_path}")

Wrote config to /Users/niccolo/Desktop/Dynamics_Networks/configs/config_Kuramoto_Bistable_Graphon.json


### Create a sweep table (for phase diagrams)
When one parameter gets changed we can use the config file as a base and then override the value from a sweep table!
This cell writes a simple TSV file you can use with `--params-table` and `--row-index`.

In [4]:
# Define the sweep values
sigma = noise["params"]["sigma"]
n = network["params"]["n"]
theta_c = sigma**2 / 0.04618426230106129

theta_values = np.concatenate([
    np.linspace(0.5 * theta_c, theta_c, 5),
    np.linspace(theta_c, 2 * theta_c, 10)[1:],
])

# Number of graph and noise realizations per theta
n_graph = 5
n_noise = 3

# Build a table with dot-path columns that override the base config
records = []
for theta_idx, theta in enumerate(theta_values, start=1):
    for g_idx in range(n_graph):
        # Unique graph per theta, reused for n_noise noise realizations.
        graph_seed = (theta_idx - 1) * n_graph + g_idx
        for n_idx in range(n_noise):
            # Unique noise seed per run.
            run_seed = (theta_idx - 1) * (n_graph * n_noise) + g_idx * n_noise + n_idx
            run_id = f"theta_{theta_idx:02d}_g{g_idx:02d}_n{n_idx:02d}"
            records.append({
                "run_id": run_id,
                "system.params.theta": float(theta),
                "network.params.seed": int(graph_seed),
                "run.seed": int(run_seed),
            })

df = pd.DataFrame(records)


In [5]:
# Output path for the sweep table
path_folder = "params"
name_sweep_table = "sweep_theta_Kuramoto_Bistable_Graphon.tsv"
sweep_path = repo_root / path_folder / name_sweep_table
sweep_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(sweep_path, sep="\t", index=False)
print(f"Wrote sweep table to {sweep_path}")

Wrote sweep table to /Users/niccolo/Desktop/Dynamics_Networks/params/sweep_theta_Kuramoto_Bistable_Graphon.tsv
